In [1]:
# Import pandas 
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import History,LearningRateScheduler
from tensorflow.keras.layers import Dropout,Dense,Activation

In [2]:
# Load data
dataset = pd.read_csv("data/training set values.csv", sep=',', error_bad_lines=False, header=0, index_col=0)
datalabel = pd.read_csv("data/training set labels.csv", sep=",", error_bad_lines=False, header=0, index_col=0)
testset = pd.read_csv("data/test set values.csv", sep=',', error_bad_lines=False, header=0, index_col=0)
finalfile_index=testset.index
dataframe = dataset.join(datalabel, on=["id"])

In [3]:
# Add some columns
dataframe['functional'] = (dataframe['status_group'] == "functional") * 1
dataframe['non_functional'] = (dataframe['status_group'] == "non functional") * 1
dataframe['functional_needs_repair'] = (dataframe['status_group'] == "functional needs repair") * 1
dataframe['categorical_population'] = pd.qcut(dataframe['population'][dataframe['population'] > 0], 4)
dataframe['categorical_gps_height'] = pd.qcut(dataframe['gps_height'][dataframe['gps_height'] > 0], 4)
dataframe['categorical_construction_year'] = pd.qcut(dataframe['construction_year'][dataframe['construction_year'] > 0], 4)
dataframe.loc[dataframe['construction_year'] == 0, 'construction_year'] = dataframe['construction_year'].min()
dataframe['categorical_usage_time'] = dataframe.apply(lambda row: int(row.date_recorded.split('-')[2]) - row.construction_year, axis=1)
dataframe['categorical_usage_time'] = pd.qcut(dataframe['categorical_usage_time'][dataframe['categorical_usage_time'] > 0], 4)

testset['categorical_population'] = pd.qcut(testset['population'][testset['population'] > 0], 4)
testset['categorical_gps_height'] = pd.qcut(testset['gps_height'][testset['gps_height'] > 0], 4)
testset['categorical_construction_year'] = pd.qcut(testset['construction_year'][testset['construction_year'] > 0], 4)
testset.loc[testset['construction_year'] == 0, 'construction_year'] = testset['construction_year'].min()
testset['categorical_usage_time'] = testset.apply(lambda row: int(row.date_recorded.split('-')[2]) - row.construction_year, axis=1)
testset['categorical_usage_time'] = pd.qcut(testset['categorical_usage_time'][testset['categorical_usage_time'] > 0], 4)

dataframe

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,waterpoint_type,waterpoint_type_group,status_group,functional,non_functional,functional_needs_repair,categorical_population,categorical_gps_height,categorical_construction_year,categorical_usage_time
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322e+00,none,0,Lake Nyasa,...,communal standpipe,communal standpipe,functional,1,0,0,"(40.0, 150.0]","(1194.0, 1512.0]","(1987.0, 2000.0]",NaN
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466e+00,Zahanati,0,Lake Victoria,...,communal standpipe,communal standpipe,functional,1,0,0,"(150.0, 324.0]","(1194.0, 1512.0]","(2008.0, 2013.0]",NaN
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329e+00,Kwa Mahundi,0,Pangani,...,communal standpipe multiple,communal standpipe,functional,1,0,0,"(150.0, 324.0]","(471.0, 1194.0]","(2008.0, 2013.0]",NaN
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-1.115530e+01,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,communal standpipe multiple,communal standpipe,non functional,0,1,0,"(40.0, 150.0]","(0.999, 471.0]","(1959.999, 1987.0]",NaN
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359e+00,Shuleni,0,Lake Victoria,...,communal standpipe,communal standpipe,functional,1,0,0,NaN,NaN,NaN,"(8.0, 15.0]"
9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587e+00,Tajiri,0,Pangani,...,communal standpipe multiple,communal standpipe,functional,1,0,0,"(0.999, 40.0]",NaN,"(2008.0, 2013.0]",NaN
19816,0.0,2012-10-01,Dwsp,0,DWSP,33.362410,-3.766365e+00,Kwa Ngomho,0,Internal,...,hand pump,hand pump,non functional,0,1,0,NaN,NaN,NaN,"(0.999, 8.0]"
54551,0.0,2012-10-09,Rwssp,0,DWE,32.620617,-4.226198e+00,Tushirikiane,0,Lake Tanganyika,...,hand pump,hand pump,non functional,0,1,0,NaN,NaN,NaN,"(8.0, 15.0]"
53934,0.0,2012-11-03,Wateraid,0,Water Aid,32.711100,-5.146712e+00,Kwa Ramadhan Musa,0,Lake Tanganyika,...,hand pump,hand pump,non functional,0,1,0,NaN,NaN,NaN,"(0.999, 8.0]"


In [4]:
# Geographic location.sort_values(by=('functional'))

print(dataframe[['region', 'functional', 'non_functional', 'functional_needs_repair']].groupby(['region']).mean().sort_values(by=('functional')))

               functional  non_functional  functional_needs_repair
region                                                            
Lindi            0.297542        0.642303                 0.060155
Mtwara           0.302890        0.624277                 0.072832
Rukwa            0.391040        0.534292                 0.074668
Tabora           0.432874        0.544155                 0.022971
Mara             0.449975        0.519553                 0.030472
Dodoma           0.458428        0.446615                 0.094957
Singida          0.483039        0.455805                 0.061156
Kigoma           0.484020        0.301847                 0.214134
Mwanza           0.484204        0.456802                 0.058994
Mbeya            0.499892        0.391464                 0.108644
Kagera           0.520808        0.387515                 0.091677
Morogoro         0.528957        0.396156                 0.074888
Shinyanga        0.559815        0.312726                 0.12

In [5]:
# How the waterpoint is managed

print(dataframe[['management', 'functional', 'non_functional', 'functional_needs_repair']].groupby(['management']).mean().sort_values(by=('functional')))

                  functional  non_functional  functional_needs_repair
management                                                           
other - school      0.232323        0.757576                 0.010101
company             0.389781        0.588321                 0.021898
unknown             0.399287        0.552585                 0.048128
water authority     0.493363        0.449115                 0.057522
vwc                 0.504234        0.426864                 0.068902
parastatal          0.576923        0.303733                 0.119344
trust               0.589744        0.333333                 0.076923
other               0.598341        0.336493                 0.065166
wug                 0.599540        0.301458                 0.099002
wua                 0.690730        0.228402                 0.080868
water board         0.739857        0.169792                 0.090351
private operator    0.748858        0.228818                 0.022324


In [6]:
# The kind of extraction the waterpoint uses

print(dataframe[['extraction_type_class', 'functional', 'non_functional', 'functional_needs_repair']].groupby(['extraction_type_class']).mean().sort_values(by=('functional')))

                       functional  non_functional  functional_needs_repair
extraction_type_class                                                     
other                    0.160031        0.807932                 0.032037
motorpump                0.379980        0.573820                 0.046200
wind-powered             0.427350        0.512821                 0.059829
submersible              0.538760        0.420295                 0.040945
gravity                  0.599253        0.299888                 0.100859
handpump                 0.630469        0.309067                 0.060464
rope pump                0.649667        0.312639                 0.037694


In [7]:
# What the water costs

print(dataframe[['payment', 'functional', 'non_functional', 'functional_needs_repair']].groupby(['payment']).mean().sort_values(by=('functional')))

                       functional  non_functional  functional_needs_repair
payment                                                                   
unknown                  0.432512        0.514527                 0.052961
never pay                0.448911        0.475856                 0.075233
other                    0.579696        0.308349                 0.111954
pay when scheme fails    0.620593        0.308636                 0.070772
pay monthly              0.660482        0.227831                 0.111687
pay per bucket           0.677796        0.276683                 0.045520
pay annually             0.752334        0.179846                 0.067820


In [8]:
# The quality of the water

print(dataframe[['quality_group', 'functional', 'non_functional', 'functional_needs_repair']].groupby(['quality_group']).mean().sort_values(by=('functional')))

               functional  non_functional  functional_needs_repair
quality_group                                                     
unknown          0.140725        0.840618                 0.018657
salty            0.460828        0.482002                 0.057170
colored          0.502041        0.387755                 0.110204
milky            0.544776        0.437811                 0.017413
good             0.565941        0.357236                 0.076823
fluoride         0.723502        0.216590                 0.059908


In [9]:
# The quantity of water

print(dataframe[['quantity', 'functional', 'non_functional', 'functional_needs_repair']]
      .groupby(['quantity'])
      .mean()
      .sort_values(by=('functional'))
 )

              functional  non_functional  functional_needs_repair
quantity                                                         
dry             0.025136        0.968940                 0.005924
unknown         0.269962        0.712294                 0.017744
insufficient    0.523234        0.380924                 0.095842
seasonal        0.574074        0.323210                 0.102716
enough          0.652323        0.275357                 0.072320


In [10]:
# The source of the water

print(dataframe[['source_type', 'functional', 'non_functional', 'functional_needs_repair']]
      .groupby(['source_type'])
      .mean()
      .sort_values(by=('functional'))
 )

                      functional  non_functional  functional_needs_repair
source_type                                                              
dam                     0.385671        0.577744                 0.036585
shallow well            0.494769        0.448348                 0.056883
borehole                0.495355        0.462131                 0.042514
river/lake              0.542257        0.338923                 0.118820
other                   0.568345        0.413669                 0.017986
rainwater harvesting    0.603922        0.259259                 0.136819
spring                  0.622290        0.302744                 0.074966


In [11]:
# The population number

print(dataframe[['categorical_population', 'functional', 'non_functional', 'functional_needs_repair']]
      .groupby(['categorical_population'])
      .mean()
      .sort_values(by=('functional'))
 )

                        functional  non_functional  functional_needs_repair
categorical_population                                                     
(0.999, 40.0]             0.515170        0.438618                 0.046213
(150.0, 324.0]            0.522978        0.404421                 0.072600
(324.0, 30500.0]          0.539612        0.376422                 0.083965
(40.0, 150.0]             0.625376        0.308378                 0.066246


In [12]:
# The gps height

print(dataframe[['categorical_gps_height', 'functional', 'non_functional', 'functional_needs_repair']]
      .groupby(['categorical_gps_height'])
      .mean()
      .sort_values(by=('functional'))
 )

                        functional  non_functional  functional_needs_repair
categorical_gps_height                                                     
(0.999, 471.0]            0.480111        0.471153                 0.048736
(471.0, 1194.0]           0.520867        0.386808                 0.092326
(1194.0, 1512.0]          0.564841        0.359483                 0.075675
(1512.0, 2770.0]          0.677361        0.271201                 0.051438


In [13]:
# The construction year

print(dataframe[['categorical_construction_year', 'functional', 'non_functional', 'functional_needs_repair']]
      .groupby(['categorical_construction_year'])
      .mean()
      .sort_values(by=('functional'))
 )

                               functional  non_functional  \
categorical_construction_year                               
(1959.999, 1987.0]               0.348167        0.573671   
(1987.0, 2000.0]                 0.549500        0.383556   
(2000.0, 2008.0]                 0.647861        0.286475   
(2008.0, 2013.0]                 0.723681        0.229789   

                               functional_needs_repair  
categorical_construction_year                           
(1959.999, 1987.0]                            0.078162  
(1987.0, 2000.0]                              0.066943  
(2000.0, 2008.0]                              0.065664  
(2008.0, 2013.0]                              0.046530  


In [14]:
# The old usage

print(dataframe[['categorical_usage_time', 'functional', 'non_functional', 'functional_needs_repair']]
      .groupby(['categorical_usage_time'])
      .mean()
      .sort_values(by=('functional'))
 )

                        functional  non_functional  functional_needs_repair
categorical_usage_time                                                     
(23.0, 31.0]              0.488067        0.416685                 0.095249
(15.0, 23.0]              0.502305        0.400148                 0.097547
(8.0, 15.0]               0.512532        0.410142                 0.077327
(0.999, 8.0]              0.531945        0.391421                 0.076633


In [15]:
# Drop some columns
dataframe.drop([
    "funder",
    "gps_height",
    "installer",
    "longitude",
    "latitude",
    "wpt_name",
    "num_private",
    "basin",
    "subvillage",
    "region_code",
    "district_code",
    "lga",
    "ward",
    "population",
    "public_meeting",
    "recorded_by",
    "scheme_management",
    "extraction_type_group",
    "management_group",
    "payment",
    "payment_type",
    "water_quality",
    "quantity_group",
    "source",
    "source_class",
    "waterpoint_type",
    "scheme_name",
    "permit",
    "construction_year",
    "extraction_type",
    "waterpoint_type_group",
    "date_recorded",
    "amount_tsh",
    "functional",
    "non_functional",
    "functional_needs_repair"
], inplace=True, axis=1)
testset.drop([
    "funder",
    "gps_height",
    "installer",
    "longitude",
    "latitude",
    "wpt_name",
    "num_private",
    "basin",
    "subvillage",
    "region_code",
    "district_code",
    "lga",
    "ward",
    "population",
    "public_meeting",
    "recorded_by",
    "scheme_management",
    "extraction_type_group",
    "management_group",
    "payment",
    "payment_type",
    "water_quality",
    "quantity_group",
    "source",
    "source_class",
    "waterpoint_type",
    "scheme_name",
    "permit",
    "construction_year",
    "extraction_type",
    "waterpoint_type_group",
    "date_recorded",
    "amount_tsh",
], inplace=True, axis=1)

# Transform non numeric values to numeric values
nonNumericKeys = [
    'region', 'management', 'extraction_type_class', 'quality_group', 'quantity', 'source_type', 
    'categorical_population', 'categorical_gps_height', 'categorical_construction_year', 'categorical_usage_time',
]
for nonNumericKey in nonNumericKeys:
    print(dataframe[nonNumericKey].unique().tolist())
    
    # Transformation
    for (i, value) in enumerate(dataframe[nonNumericKey].unique().tolist()):
        dataframe[nonNumericKey].replace(value, i, inplace=True)
    for (i, value) in enumerate(testset[nonNumericKey].unique().tolist()):
        testset[nonNumericKey].replace(value, i, inplace=True)
    
    # Convert column to int
    dataframe[nonNumericKey] = dataframe[nonNumericKey].astype(int)
    testset[nonNumericKey] = testset[nonNumericKey].astype(int)
    
# Transformation for status_group
# Convert column to int
dataframe['status_group'].replace('functional', 1, inplace=True)
dataframe['status_group'].replace('functional needs repair', 0, inplace=True)
dataframe['status_group'].replace('non functional', -1, inplace=True)
dataframe['status_group'] = dataframe['status_group'].astype(int)

dataframe = dataframe.reindex(columns=(['status_group'] + list([a for a in dataframe.columns if a != 'status_group']) ))
    
dataframe

['Iringa', 'Mara', 'Manyara', 'Mtwara', 'Kagera', 'Tanga', 'Shinyanga', 'Tabora', 'Pwani', 'Ruvuma', 'Kilimanjaro', 'Rukwa', 'Mwanza', 'Kigoma', 'Lindi', 'Dodoma', 'Arusha', 'Mbeya', 'Singida', 'Morogoro', 'Dar es Salaam']
['vwc', 'wug', 'other', 'private operator', 'water board', 'wua', 'company', 'water authority', 'parastatal', 'unknown', 'other - school', 'trust']
['gravity', 'submersible', 'handpump', 'other', 'motorpump', 'wind-powered', 'rope pump']
['good', 'salty', 'milky', 'unknown', 'fluoride', 'colored']
['enough', 'insufficient', 'dry', 'seasonal', 'unknown']
['spring', 'rainwater harvesting', 'dam', 'borehole', 'other', 'shallow well', 'river/lake']
[Interval(40.0, 150.0, closed='right'), Interval(150.0, 324.0, closed='right'), nan, Interval(0.999, 40.0, closed='right'), Interval(324.0, 30500.0, closed='right')]
[Interval(1194.0, 1512.0, closed='right'), Interval(471.0, 1194.0, closed='right'), Interval(0.999, 471.0, closed='right'), nan, Interval(1512.0, 2770.0, closed='

,status_group,region,extraction_type_class,management,quality_group,quantity,source_type,categorical_population,categorical_gps_height,categorical_construction_year,categorical_usage_time
id,,,,,,,,,,,
69572,1,0,0,0,0,0,0,0,0,0,0
8776,1,1,0,1,0,1,1,1,0,1,0
34310,1,2,0,0,0,0,2,1,1,1,0
67743,-1,3,1,0,0,2,3,0,2,2,0
19728,1,4,0,2,0,3,1,2,3,3,1
9944,1,5,1,0,1,0,4,3,3,1,0
19816,-1,6,2,0,0,0,3,2,3,3,2
54551,-1,6,2,1,2,0,5,2,3,3,1
53934,-1,7,2,0,1,3,3,2,3,3,2


In [16]:
from sklearn.model_selection import train_test_split
#Séparation des valeurs de train et label (tous les exemples)
X_alltrain = dataframe.values[:, 1:]
y_alltrain = dataframe.values[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X_alltrain, y_alltrain, random_state=42)
print('%i X_train, %i X_test, %i y_train,%i y_test'%(
    X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]))
print('%s X_alltrain, %s y_alltrain'%(X_alltrain.shape, y_alltrain.shape))
print(y_alltrain)
feature_names=dataframe.columns.tolist()[1:]
target_names=["Non fonctionnel","Fonctionnel"]
print('features:',feature_names)
print('target:',target_names)

44550 X_train, 14850 X_test, 44550 y_train,14850 y_test
(59400, 10) X_alltrain, (59400,) y_alltrain
[1 1 1 ... 1 1 1]
features: ['region', 'extraction_type_class', 'management', 'quality_group', 'quantity', 'source_type', 'categorical_population', 'categorical_gps_height', 'categorical_construction_year', 'categorical_usage_time']
target: ['Non fonctionnel', 'Fonctionnel']


In [17]:
# Fonctions permettant de générer le fichier d'envoi à Kaggle
#
#parametres: Classifiers; Données à calculer ; index)
def generer_resultats(clf,data=dataframe.values):
    """
    Fonctions permettant de générer le fichier d'envoi à Kaggle.
    On passe un classifier sur lequel on refait le training avec toutes les données de training
    
    Parameters
    ----------
    Classifiers : Classifier utilisé pour la prédiction
    data : Données à calculer. par défaut, les valeurs du dataset "test"
    """ 
    print(clf.get_params())
    clf.fit(X_alltrain, y_alltrain)
    prediction=clf.predict(data)
    results=pd.DataFrame(prediction.astype(int), index = finalfile_index, columns=['status_group'])
    results['status_group'].replace(1, 'functional', inplace=True)
    results['status_group'].replace(0, 'functional needs repair', inplace=True)
    results['status_group'].replace(-1, 'non functional', inplace=True)
    results.to_csv('resultats%s.csv'%clf.__class__.__name__)
    
#Fonction pour l'affichage 2 D des résultats    
from matplotlib.colors import ListedColormap
def plot_decision_boundary(clf,X,y, axes=[-0, 30, -5, 5], axis_name=['x1','x2'],alpha=0.5, contour=True):
    """
    Fonction pour l'affichage 2 D des résultats   
    
    Parameters
    ----------
    clf : Classifier à afficher
    X : features de Données a afficher
    y : labels de Données a afficher  
    axes : : Tailles des axes (valeur min/max)
    axis_name : Nom des axes sur le graphique
    alpha : Transparence des points
    contour : Afichage du contour
    """     
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if contour:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bo",label="Non fonctionnel", alpha=alpha)
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "ys", label="Fonctionnel",alpha=alpha)
    plt.axis(axes)
    plt.xlabel(axis_name[0], fontsize=18)
    plt.ylabel(axis_name[1]+ "  ",fontsize=18, rotation=0)    
    plt.legend(loc="lower right", fontsize=14)

In [18]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

gnb_clf = RandomForestClassifier(random_state=2)
y_train = y_train.astype("int")
gnb_clf.fit(X_train, y_train)
y_pred = gnb_clf.predict(X_test)    
print("Niveau de précision : %.2f"%(100*accuracy_score(y_test, y_pred)))

/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Niveau de précision : 75.06


In [19]:
# Graphes
%matplotlib inline
import matplotlib.pyplot as plt

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

#Eviter les warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

rnd_clf = RandomForestClassifier(random_state=2)

# Lance sur les données de test
generer_resultats(rnd_clf, testset.values)

{'oob_score': False, 'warm_start': False, 'min_samples_split': 2, 'min_impurity_split': None, 'n_jobs': None, 'random_state': 2, 'max_leaf_nodes': None, 'bootstrap': True, 'min_weight_fraction_leaf': 0.0, 'max_features': 'auto', 'class_weight': None, 'n_estimators': 'warn', 'verbose': 0, 'criterion': 'gini', 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'max_depth': None}
